<a href="https://colab.research.google.com/github/simaryadav/learning_roadmap/blob/main/Bidirectional_lstm_fake_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.model_selection import train_test_split


In [4]:
df = pd.read_csv("train.csv", on_bad_lines='skip', engine='python')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull().sum()

,0
id,0
title,310
author,1064
text,23
label,0


In [8]:
df = df.dropna()

In [9]:
df.isnull().sum()

,0
id,0
title,0
author,0
text,0
label,0


In [16]:
y = df["label"]
x = df.drop('label',axis = 1)

In [17]:
y.value_counts()

,count
label,
0,5669
1,4354


In [18]:
tf.__version__

'2.17.0'

In [21]:
voc_size = 6000

In [22]:
messages = x.copy()
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
11391,11391,"Within Hours Of Donald Trump Being President, ...",Starkman,Donald Trump won’t officially become president...
11392,11392,Three Arrested for Starting Massive Fire on At...,Katherine Rodriguez,Three people were arrested Friday in connectio...
11393,11393,Oh My… WikiLeaks Email Between Stratfor Intel ...,LN Buzz Team,Vince Foster is a name most of us probably hav...
11395,11395,Comment on Censored News: Black woman kills 12...,TNB – PA,"Posted on August 16, 2013 by Dr. Eowyn | 143 C..."


In [25]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re,nltk
from nltk.stem import WordNetLemmatizer

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
!pip install nltk # install nltk if you haven't already
import nltk # imports nltk
nltk.download('wordnet') # downloads wordnet

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [49]:
messages.reset_index(inplace = True)

In [50]:
ws = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)): # The error is likely in this loop as 'i' iterates from 0 to the length of messages
  print("processing at ", i)
  try: # This will try to run the code
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # Access the 'title' column at index i. If index i does not exist, this will cause an error.
  except KeyError: # If a KeyError occurs, this except block will run
    print(f"Skipping index {i} as it does not exist") # Prints a message to inform the user that a message was skipped
    continue # This will skip to the next iteration of the loop
  review = review.lower()
  review = review.split()
  review = [ws.lemmatize(word) for word in review] # Lemmatize each word
  review = ' '.join(review) # Join the words back into a string
  corpus.append(review) # Add the processed review to the corpus

Streaming output truncated to the last 5000 lines.
processing at  5023
processing at  5024
processing at  5025
processing at  5026
processing at  5027
processing at  5028
processing at  5029
processing at  5030
processing at  5031
processing at  5032
processing at  5033
processing at  5034
processing at  5035
processing at  5036
processing at  5037
processing at  5038
processing at  5039
processing at  5040
processing at  5041
processing at  5042
processing at  5043
processing at  5044
processing at  5045
processing at  5046
processing at  5047
processing at  5048
processing at  5049
processing at  5050
processing at  5051
processing at  5052
processing at  5053
processing at  5054
processing at  5055
processing at  5056
processing at  5057
processing at  5058
processing at  5059
processing at  5060
processing at  5061
processing at  5062
processing at  5063
processing at  5064
processing at  5065
processing at  5066
processing at  5067
processing at  5068
processing at  5069
processin

In [51]:
corpus

['house dem aide we didn t even see comey s letter until jason chaffetz tweeted it',
 'flynn hillary clinton big woman on campus breitbart',
 'why the truth might get you fired',
 'civilian killed in single u airstrike have been identified',
 'iranian woman jailed for fictional unpublished story about woman stoned to death for adultery',
 'jackie mason hollywood would love trump if he bombed north korea over lack of trans bathroom exclusive video breitbart',
 'beno t hamon win french socialist party s presidential nomination the new york time',
 'a back channel plan for ukraine and russia courtesy of trump associate the new york time',
 'obama s organizing for action partner with soros linked indivisible to disrupt trump s agenda',
 'bbc comedy sketch real housewife of isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter in the arctic photo',
 'u official see no link between trump and russia',
 're yes there are paid government troll on social med

In [52]:
onehot_repr =  [one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[663, 4949, 4371, 2389, 1726, 24, 4511, 1715, 4863, 4535, 5821, 2181, 4946, 5422, 231, 5936], [4024, 4483, 467, 4905, 4483, 3823, 1799, 5061], [2522, 4125, 113, 5980, 5078, 3869, 5315], [5665, 4375, 542, 5185, 3971, 5349, 2343, 3885, 1747], [3155, 4483, 2132, 489, 2036, 1883, 3448, 4230, 4483, 4610, 1684, 5321, 489, 5527], [4426, 3189, 5068, 4914, 1712, 4604, 4409, 38, 4775, 4852, 2675, 2242, 2718, 2702, 1750, 1955, 3688, 5606, 5061], [1147, 24, 4713, 4329, 5241, 4813, 3903, 4535, 3114, 630, 4125, 4235, 731, 4364], [116, 2167, 4903, 760, 489, 544, 4850, 1583, 3421, 2702, 4604, 3885, 4125, 4235, 731, 4364], [2800, 4535, 1228, 489, 2598, 1783, 5560, 4576, 1785, 4617, 1684, 4939, 4604, 4535, 93], [1744, 2711, 632, 4993, 3963, 2702, 4722, 5151, 5753], [543, 2566, 1893, 1685, 5495, 58, 5666, 1370, 5863, 542, 4125, 2296, 2242], [3971, 992, 1715, 4065, 1588, 3909, 4604, 4850, 1583], [2178, 4690, 4090, 2151, 2352, 4501, 1070, 3823, 4308, 894, 1774, 4629, 4850, 5549], [542, 3903, 4583, 1596, 2

In [53]:
sent_length = 30
embed_docs = pad_sequences(onehot_repr,padding = 'pre',maxlen = sent_length)

array([[   0,    0,    0, ..., 5422,  231, 5936],
       [   0,    0,    0, ..., 3823, 1799, 5061],
       [   0,    0,    0, ..., 5078, 3869, 5315],
       ...,
       [   0,    0,    0, ..., 4235,  731, 4364],
       [   0,    0,    0, ..., 4364,  915, 1856],
       [   0,    0,    0, ..., 4235,  731, 4364]], dtype=int32)

In [54]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
print(model.summary)

<bound method Model.summary of <Sequential name=sequential_2, built=False>>


In [55]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [56]:
x_final = np.array(embed_docs)
y_final = np.array(y)

In [57]:
x_final.shape,y_final.shape

((10023, 30), (10023,))

In [58]:
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

In [60]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 10,batch_size = 64)

Epoch 1/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 16s 99ms/step - accuracy: 0.7134 - loss: 0.5239 - val_accuracy: 0.8933 - val_loss: 0.2547
Epoch 2/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 109ms/step - accuracy: 0.9264 - loss: 0.1836 - val_accuracy: 0.9181 - val_loss: 0.2081
Epoch 3/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 107ms/step - accuracy: 0.9691 - loss: 0.0869 - val_accuracy: 0.9187 - val_loss: 0.2003
Epoch 4/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 108ms/step - accuracy: 0.9825 - loss: 0.0622 - val_accuracy: 0.9187 - val_loss: 0.2365
Epoch 5/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 114ms/step - accuracy: 0.9945 - loss: 0.0259 - val_accuracy: 0.9078 - val_loss: 0.3253
Epoch 6/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 20s 106ms/step - accuracy: 0.9967 - loss: 0.0122 - val_accuracy: 0.9111 - val_loss: 0.3065
Epoch 7/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 20s 98ms/step - accuracy: 0.9989 - loss: 0.0083 - val_accuracy: 0.9117 - val_loss: 0.3477
Epoch 8/10
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 106ms/step - accuracy: 0.9996 - loss: 0.0

In [61]:
y_pred = model.predict(X_test)

104/104 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step


In [62]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [66]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[1713,  182],
       [ 116, 1297]])